In [1]:
import pandas as pd
import talib


import yfinance as yf
import datetime as dt
import pandas_ta as pta
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, Conv2D, Flatten, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [2]:
import sys

pth_apple = "/Users/josesanchez/Desktop/Code/GitHub/StockPrediction/FileBase"
pth_win = "/Users/Usuario/Desktop/GitHub/StockPrediction/FileBase"
sys.path.append(pth_win)


from root_functions import add_indicators, up_down, full_prediction, full_prediction_binary, result, aprox_beneficios

In [3]:
load_model = load_model('saved_model/my_model')
load_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 32)             3488      
_________________________________________________________________
lstm (LSTM)                  (None, 1, 128)            82432     
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 128)            512       
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 256)            394240    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 256)            1024      
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 128)            197120    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 128)            5

## Descarga datos y creación dataset

In [4]:
sample_per = 10000
stock = 'EURGBP=X'

end = dt.date.today()
start = end - dt.timedelta(days=(sample_per))
interval = '1d'

In [5]:
database = yf.download(stock, start=start, end=end, interval=interval)

[*********************100%***********************]  1 of 1 completed


## Add Indicators

In [6]:
#Añadimos indicadores

ema1=200
ema2=50
ema3=20

data_base = add_indicators(database)

#Añadimos columna up_down
data_base = up_down(data_base)

#Listado indicadores
data_base.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '200ema',
       '50ema', '20ema', '14-high',
       ...
       'CDL_STICKSANDWICH', 'CDL_TAKURI', 'CDL_TASUKIGAP', 'CDL_THRUSTING',
       'CDL_TRISTAR', 'CDL_UNIQUE3RIVER', 'CDL_UPSIDEGAP2CROWS',
       'CDL_XSIDEGAP3METHODS', 'log_ret', 'Up_Down'],
      dtype='object', length=109)

In [7]:
data_base.columns.values

array(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '200ema',
       '50ema', '20ema', '14-high', '14-low', '%K', '%D', 'RSI', 'MACD',
       'MACDh', 'MACDs', 'CCI', 'PSAR', 'BBl', 'BBm', 'BBu', 'BBb', 'BBp',
       'ATR', 'ADX', 'ADX_DMP', 'ADX_DMN', 'APO', 'AROON_D', 'AROON_U',
       'AROON_OSC', 'BOP', 'CMO', 'MOM', 'PPO', 'PPO_h', 'PPO_s', 'ROC',
       'STOCHRSI_k', 'STOCHRSI_d', 'TRIX', 'TRIX_s', 'WILLR', 'NATR',
       'CDL_2CROWS', 'CDL_3BLACKCROWS', 'CDL_3INSIDE', 'CDL_3LINESTRIKE',
       'CDL_3OUTSIDE', 'CDL_3STARSINSOUTH', 'CDL_3WHITESOLDIERS',
       'CDL_ABANDONEDBABY', 'CDL_ADVANCEBLOCK', 'CDL_BELTHOLD',
       'CDL_BREAKAWAY', 'CDL_CLOSINGMARUBOZU', 'CDL_CONCEALBABYSWALL',
       'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER', 'CDL_DOJI_10_0.1',
       'CDL_DOJISTAR', 'CDL_DRAGONFLYDOJI', 'CDL_ENGULFING',
       'CDL_EVENINGDOJISTAR', 'CDL_EVENINGSTAR', 'CDL_GAPSIDESIDEWHITE',
       'CDL_GRAVESTONEDOJI', 'CDL_HAMMER', 'CDL_HANGINGMAN', 'CDL_HARAMI',
       'CDL_

In [30]:
#Hacemos un shift a la columna Up_Down para tener el signo de la tendencia de la siguiente vela
data_base['Up_Down'] = data_base['Up_Down'].shift(-1)

data_base.dropna(inplace=True)

In [31]:
#Filtramos las columnas que nos interesan para obtener el dataset de trabajo
db = data_base.copy()


data = db[['Up_Down','log_ret','Open', 'High', 'Low', 'Close', 'Adj Close','Volume', '200ema',
       '50ema', '20ema', '14-high', '14-low', '%K', '%D', 'RSI', 'MACD',
       'MACDh', 'MACDs', 'CCI', 'PSAR', 'BBl', 'BBm', 'BBu', 'BBb', 'BBp',
       'ATR', 'ADX', 'ADX_DMP', 'ADX_DMN', 'APO', 'AROON_D', 'AROON_U',
       'AROON_OSC', 'BOP', 'CMO', 'MOM', 'PPO', 'PPO_h', 'PPO_s', 'ROC',
       'STOCHRSI_k', 'STOCHRSI_d', 'TRIX', 'TRIX_s', 'WILLR', 'NATR',
       'CDL_2CROWS', 'CDL_3BLACKCROWS', 'CDL_3INSIDE', 'CDL_3LINESTRIKE',
       'CDL_3OUTSIDE', 'CDL_3STARSINSOUTH', 'CDL_3WHITESOLDIERS',
       'CDL_ABANDONEDBABY', 'CDL_ADVANCEBLOCK', 'CDL_BELTHOLD',
       'CDL_BREAKAWAY', 'CDL_CLOSINGMARUBOZU', 'CDL_CONCEALBABYSWALL',
       'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER', 'CDL_DOJI_10_0.1',
       'CDL_DOJISTAR', 'CDL_DRAGONFLYDOJI', 'CDL_ENGULFING',
       'CDL_EVENINGDOJISTAR', 'CDL_EVENINGSTAR', 'CDL_GAPSIDESIDEWHITE',
       'CDL_GRAVESTONEDOJI', 'CDL_HAMMER', 'CDL_HANGINGMAN', 'CDL_HARAMI',
       'CDL_HARAMICROSS', 'CDL_HIGHWAVE', 'CDL_HIKKAKE', 'CDL_HIKKAKEMOD',
       'CDL_HOMINGPIGEON', 'CDL_IDENTICAL3CROWS', 'CDL_INNECK',
       'CDL_INSIDE', 'CDL_INVERTEDHAMMER', 'CDL_KICKING',
       'CDL_KICKINGBYLENGTH', 'CDL_LADDERBOTTOM', 'CDL_LONGLEGGEDDOJI',
       'CDL_LONGLINE', 'CDL_MARUBOZU', 'CDL_MATCHINGLOW', 'CDL_MATHOLD',
       'CDL_MORNINGDOJISTAR', 'CDL_MORNINGSTAR', 'CDL_ONNECK',
       'CDL_PIERCING', 'CDL_RICKSHAWMAN', 'CDL_RISEFALL3METHODS',
       'CDL_SEPARATINGLINES', 'CDL_SHOOTINGSTAR', 'CDL_SHORTLINE',
       'CDL_SPINNINGTOP', 'CDL_STALLEDPATTERN', 'CDL_STICKSANDWICH',
       'CDL_TAKURI', 'CDL_TASUKIGAP', 'CDL_THRUSTING', 'CDL_TRISTAR',
       'CDL_UNIQUE3RIVER', 'CDL_UPSIDEGAP2CROWS', 'CDL_XSIDEGAP3METHODS']]


cols = data.columns

In [32]:
data.reset_index(inplace=True)
data.drop(columns=data.columns[0], inplace=True)

## Preparacion datos para el LSTM

In [33]:
data

,Up_Down,log_ret,Open,High,Low,Close,Adj Close,Volume,200ema,50ema,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
0,1.0,-0.000910,0.65910,0.66070,0.65620,0.65850,0.65850,0,0.673729,0.665763,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.002733,0.65840,0.66090,0.65580,0.66030,0.66030,0,0.673515,0.665549,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,-0.000151,0.65980,0.66180,0.65820,0.66020,0.66020,0,0.673304,0.665339,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.001212,0.66000,0.66140,0.65760,0.66100,0.66100,0,0.673110,0.665169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.003933,0.65990,0.66390,0.65830,0.66360,0.66360,0,0.672961,0.665108,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5787,0.0,-0.001849,0.85816,0.85928,0.85623,0.85811,0.85811,0,0.866571,0.855628,...,-100.0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0
5788,1.0,-0.000827,0.85762,0.85856,0.85660,0.85740,0.85740,0,0.866480,0.855697,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5789,1.0,0.000467,0.85794,0.86128,0.85750,0.85780,0.85780,0,0.866393,0.855780,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
5790,0.0,0.001900,0.85944,0.86017,0.85770,0.85943,0.85943,0,0.866324,0.855923,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Convertimos todas las columnas a valor numerico
data = data.apply(pd.to_numeric)


#Escalamos los valores con un MinMaxScaler

#data['200ema'] = data['200ema'].pct_change()
#data['50ema'] = data['50ema'].pct_change()
#data['20ema'] = data['20ema'].pct_change()

data.dropna(inplace=True)

values = data.values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

values = scaled

In [25]:
values.shape

(5793, 108)

In [13]:
#Dividimos los datos entre train y test
train_X, val = train_test_split(values, test_size=0.2, shuffle=False)

#Dividimos datos validation entre validation y test
val_X, test_X = train_test_split(val, test_size=0.5, shuffle=False)

#Generamos el reshape para poder incluir los valores en el modelo LSTM
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, val_X.shape, test_X.shape)

(4634, 1, 108) (579, 1, 108) (580, 1, 108)


In [23]:
test_X.shape

(580, 1, 108)

## Resultados con valores de test

In [15]:
# make a prediction
yhat = load_model.predict(test_X)

In [24]:
yhat.shape

(580, 1, 1)

In [17]:
yhat_re = yhat.reshape((yhat.shape[0], yhat.shape[1]))

In [26]:
yhat_re.shape

(580, 1)

In [19]:
test_X_re = test_X.reshape((test_X.shape[0], test_X.shape[2]))
inv_yhat = np.concatenate((yhat_re, test_X_re), axis=1)

In [22]:
inv_yhat.shape

(580, 109)

In [20]:
inv_yhat = scaler.inverse_transform(inv_yhat)





ValueError: operands could not be broadcast together with shapes (580,109) (108,) (580,109) 